In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

### Data preprocessing

In [2]:
def load_file(path):
    return pd.read_csv(path, delim_whitespace=True, header=None)
X_train=[]
X_train.append(load_file("./UCI HAR dataset/UCI HAR dataset/train/Inertial Signals/body_acc_x_train.txt").to_numpy())
X_train.append(load_file("./UCI HAR dataset/UCI HAR dataset/train/Inertial Signals/body_acc_y_train.txt").to_numpy())
X_train.append(load_file("./UCI HAR dataset/UCI HAR dataset/train/Inertial Signals/body_acc_z_train.txt").to_numpy())
X_train.append(load_file("./UCI HAR dataset/UCI HAR dataset/train/Inertial Signals/body_gyro_x_train.txt").to_numpy())
X_train.append(load_file("./UCI HAR dataset/UCI HAR dataset/train/Inertial Signals/body_gyro_y_train.txt").to_numpy())
X_train.append(load_file("./UCI HAR dataset/UCI HAR dataset/train/Inertial Signals/body_gyro_z_train.txt").to_numpy())
X_train.append(load_file("./UCI HAR dataset/UCI HAR dataset/train/Inertial Signals/total_acc_x_train.txt").to_numpy())
X_train.append(load_file("./UCI HAR dataset/UCI HAR dataset/train/Inertial Signals/total_acc_y_train.txt").to_numpy())
X_train.append(load_file("./UCI HAR dataset/UCI HAR dataset/train/Inertial Signals/total_acc_z_train.txt").to_numpy())
X_train=np.transpose(X_train, (1,2,0))

In [3]:
X_test=[]
X_test.append(load_file("./UCI HAR dataset/UCI HAR dataset/test/Inertial Signals/body_acc_x_test.txt").to_numpy())
X_test.append(load_file("./UCI HAR dataset/UCI HAR dataset/test/Inertial Signals/body_acc_y_test.txt").to_numpy())
X_test.append(load_file("./UCI HAR dataset/UCI HAR dataset/test/Inertial Signals/body_acc_z_test.txt").to_numpy())
X_test.append(load_file("./UCI HAR dataset/UCI HAR dataset/test/Inertial Signals/body_gyro_x_test.txt").to_numpy())
X_test.append(load_file("./UCI HAR dataset/UCI HAR dataset/test/Inertial Signals/body_gyro_y_test.txt").to_numpy())
X_test.append(load_file("./UCI HAR dataset/UCI HAR dataset/test/Inertial Signals/body_gyro_z_test.txt").to_numpy())
X_test.append(load_file("./UCI HAR dataset/UCI HAR dataset/test/Inertial Signals/total_acc_x_test.txt").to_numpy())
X_test.append(load_file("./UCI HAR dataset/UCI HAR dataset/test/Inertial Signals/total_acc_y_test.txt").to_numpy())
X_test.append(load_file("./UCI HAR dataset/UCI HAR dataset/test/Inertial Signals/total_acc_z_test.txt").to_numpy())
X_test=np.transpose(X_test, (1,2,0))

In [4]:
y_train=pd.read_csv("./UCI HAR dataset/UCI HAR dataset/train/y_train.txt", delim_whitespace=True, header=None)
y_test=pd.read_csv("./UCI HAR dataset/UCI HAR dataset/test/y_test.txt", delim_whitespace=True, header=None)

In [5]:
np.shape(y_train)

(7352, 1)

In [6]:
np.shape(X_train)

(7352, 128, 9)

In [7]:
activity_labels=pd.read_csv('./UCI HAR dataset/UCI HAR dataset/activity_labels.txt', delim_whitespace=True, header=None)

In [8]:
dict(activity_labels.values)

{1: 'WALKING',
 2: 'WALKING_UPSTAIRS',
 3: 'WALKING_DOWNSTAIRS',
 4: 'SITTING',
 5: 'STANDING',
 6: 'LAYING'}

In [9]:
y_train[0]=y_train[0].map(dict(activity_labels.values))
y_test[0]=y_test[0].map(dict(activity_labels.values))
y_train.columns=['Activity']
y_test.columns=['Activity']

In [10]:
y_train

,Activity
0,STANDING
1,STANDING
2,STANDING
3,STANDING
4,STANDING
...,...
7347,WALKING_UPSTAIRS
7348,WALKING_UPSTAIRS
7349,WALKING_UPSTAIRS
7350,WALKING_UPSTAIRS


In [11]:
encoder=OneHotEncoder(sparse_output=False)

In [12]:
y_train=encoder.fit_transform(y_train)
y_test=encoder.fit_transform(y_test)

In [13]:
type(y_train)

numpy.ndarray

In [14]:
sc=StandardScaler()
X_train=sc.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test=sc.fit_transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

In [15]:
np.shape(X_train)

(7352, 128, 9)

In [16]:
np.shape(X_test)

(2947, 128, 9)

### Building Model

In [17]:
n_timesteps, n_features, n_outputs=X_train.shape[1], X_train.shape[2], y_train.shape[1]

In [18]:
n_outputs

6

In [19]:
n_timesteps

128

In [20]:
n_features

9

In [21]:
conv1d_model=tf.keras.Sequential()
conv1d_model.add(tf.keras.layers.Conv1D(filters=70, kernel_size=3, activation='relu', input_shape=(n_timesteps, n_features), padding='same'))
conv1d_model.add(tf.keras.layers.Conv1D(filters=70, kernel_size=3, activation='relu', padding='same'))
conv1d_model.add(tf.keras.layers.Dropout(rate=0.5))
conv1d_model.add(tf.keras.layers.MaxPooling1D(pool_size=1))
conv1d_model.add(tf.keras.layers.Flatten())
conv1d_model.add(tf.keras.layers.Dense(150, activation='relu'))
conv1d_model.add(tf.keras.layers.Dense(n_outputs, activation='softmax'))

conv1d_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
conv1d_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 128, 70)           1960      
                                                                 
 conv1d_1 (Conv1D)           (None, 128, 70)           14770     
                                                                 
 dropout (Dropout)           (None, 128, 70)           0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 128, 70)          0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 8960)              0         
                                                                 
 dense (Dense)               (None, 150)               1344150   
                                                        

In [23]:
training_history=conv1d_model.fit(X_train, y_train, epochs=50, batch_size=32)

Epoch 1/50
230/230 [==============================] - 19s 25ms/step - loss: 0.2984 - accuracy: 0.8794
Epoch 2/50
230/230 [==============================] - 2s 10ms/step - loss: 0.1176 - accuracy: 0.9472
Epoch 3/50
230/230 [==============================] - 2s 9ms/step - loss: 0.0938 - accuracy: 0.9596
Epoch 4/50
230/230 [==============================] - 2s 9ms/step - loss: 0.0965 - accuracy: 0.9580
Epoch 5/50
230/230 [==============================] - 2s 9ms/step - loss: 0.0834 - accuracy: 0.9614
Epoch 6/50
230/230 [==============================] - 2s 9ms/step - loss: 0.0688 - accuracy: 0.9661
Epoch 7/50
230/230 [==============================] - 2s 9ms/step - loss: 0.0673 - accuracy: 0.9690
Epoch 8/50
230/230 [==============================] - 2s 9ms/step - loss: 0.0598 - accuracy: 0.9737
Epoch 9/50
230/230 [==============================] - 2s 8ms/step - loss: 0.0754 - accuracy: 0.9659
Epoch 10/50
230/230 [==============================] - 2s 9ms/step - loss: 0.0564 - accuracy: 0.9

In [24]:
_, accuracy=conv1d_model.evaluate(X_test, y_test, batch_size=32)

93/93 [==============================] - 1s 4ms/step - loss: 1.0872 - accuracy: 0.9203


In [25]:
accuracy

0.9202578663825989

In [26]:
def model_maker(X_train, y_train, X_test, y_test):
    n_timesteps, n_features, n_outputs=X_train.shape[1], X_train.shape[2], y_train.shape[1]

    conv1d_model=tf.keras.Sequential()
    conv1d_model.add(tf.keras.layers.Conv1D(filters=70, kernel_size=3, activation='relu', input_shape=(n_timesteps, n_features), padding='same'))
    conv1d_model.add(tf.keras.layers.Conv1D(filters=70, kernel_size=3, activation='relu', padding='same'))
    conv1d_model.add(tf.keras.layers.Dropout(rate=0.5))
    conv1d_model.add(tf.keras.layers.MaxPooling1D(pool_size=1))
    conv1d_model.add(tf.keras.layers.Flatten())
    conv1d_model.add(tf.keras.layers.Dense(150, activation='relu'))
    conv1d_model.add(tf.keras.layers.Dense(n_outputs, activation='softmax'))

    conv1d_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    training_history=conv1d_model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)
    _, accuracy=conv1d_model.evaluate(X_test, y_test, batch_size=32)
    return accuracy

In [27]:
def summarize_results(scores):
    print(scores)
    m, s = np.mean(scores), np.std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [28]:
def experiment(num_of_repeats, X_train, y_train, X_test, y_test):
    scores = list()
    for n in range(num_of_repeats):
        score = model_maker(X_train, y_train, X_test, y_test)
        score = score * 100.0
        print('>#%d: %.3f' % (n+1, score))
        scores.append(score)
    summarize_results(scores)

In [29]:
experiment(10, X_train, y_train, X_test, y_test)

93/93 [==============================] - 0s 2ms/step - loss: 0.8228 - accuracy: 0.9243
>#1: 92.433
93/93 [==============================] - 0s 2ms/step - loss: 0.9167 - accuracy: 0.9216
>#2: 92.162
93/93 [==============================] - 0s 2ms/step - loss: 1.0886 - accuracy: 0.9223
>#3: 92.229
93/93 [==============================] - 0s 2ms/step - loss: 0.9654 - accuracy: 0.9203
>#4: 92.026
93/93 [==============================] - 0s 2ms/step - loss: 1.1443 - accuracy: 0.9114
>#5: 91.144
93/93 [==============================] - 0s 2ms/step - loss: 1.1817 - accuracy: 0.9182
>#6: 91.822
93/93 [==============================] - 0s 2ms/step - loss: 1.3549 - accuracy: 0.9223
>#7: 92.229
93/93 [==============================] - 0s 2ms/step - loss: 1.1838 - accuracy: 0.9430
>#8: 94.299
93/93 [==============================] - 0s 2ms/step - loss: 0.9929 - accuracy: 0.9284
>#9: 92.840
93/93 [==============================] - 0s 2ms/step - loss: 1.2270 - accuracy: 0.9067
>#10: 90.668
[92.43298

In [31]:
conv1d_model.save('./1D CNN Model/1D_CNN_model.h5')
conv1d_model.save_weights("./1D CNN Model/1D_CNN_model_weights")